# 通过按键控制拍照和录像
本章教程通过在页面上增加按键来控制摄像头拍照和录像，和之前的教程类似，图片会被默认保存在`static`文件夹内，视频会被默认保存在`videos`文件夹内。

## 准备工作
由于产品开机默认会自动运行主程序，主程序会占用摄像头资源，这种情况下是不能使用本教程的，需要结束主程序或禁止主程序自动运行后再重新启动机器人。

这里需要注意的是，由于机器人主程序中使用了多线程且由 crontab 配置开机自动运行，所以常规的 sudo killall python 的方法通常是不起作用的，所以我们这里介绍禁用主程序自动运行的方法。

如果你已经禁用了机器人主程序的开机自动运行，则不需要执行下面的`结束主程序`章节。

### 结束主程序
1. 点击上方本页面选项卡旁边的 “+”号，会打开一个新的名为 Launcher 的选项卡。
2. 点击 Other 内的 Terminal，打开终端窗口。
3. 在终端窗口内输入 `bash` 后按回车。
4. 现在你可以使用 Bash Shell 来控制机器人了。
5. 输入命令： `crontab -e`。
6. 如果询问希望使用什么编辑器，输入 `1` 后按回车，选择使用 nano。
7. 打开 crontab 的配置文件后，你可以看到以下两行内容
> @reboot ~/ugv_pt_rpi/ugv-env/bin/python ~/ugv_pt_rpi/app.py >> ~/ugv.log 2>&1
>
> @reboot /bin/bash ~/ugv_pt_rpi/start_jupyter.sh >> ~/jupyter_log.log 2>&1

8. 在 `……app.py >> ……` 这行的最前面添加一个 `#` 号来注释掉这行。
> #@reboot ~/ugv_pt_rpi/ugv-env/bin/python ~/ugv_pt_rpi/app.py >> ~/ugv.log 2>&1
>
>  @reboot /bin/bash ~/ugv_pt_rpi/start_jupyter.sh >> ~/jupyter_log.log 2>&1

10. 在终端页面，按 Ctrl + X 退出，它会询问你 `Save modified buffer?` 输入 `Y`，按回车，保存变更。
11. 重启设备，注意该过程会暂时关闭当前的 jupyter Lab，如果你上一步没有注释掉 `……start_jupyter.sh >>……` 这一行，那么当机器人重新开机后，你仍然可以正常使用 jupyter Lab (JupyterLab 与 机器人主程序 app.py 是互相独立运行的)，可能需要重新刷新页面。
12. 这里需要注意一点，由于下位机持续通过串口与上位机通信，上位机在重启过程中有可能会由于串口电平的连续变化不能正常开机，拿上位机为树莓派的情况举例，重启时树莓派关机后不会再开机，红灯常亮绿灯不亮，此时可以关闭机器人电源开关，再打开，机器人就能够正常重启了。
13. 输入重启命令： `sudo reboot`
14. 等待设备重启后（重启过程中树莓派的绿灯会闪烁，当绿灯闪烁频率降低或灭掉后即代表已经启动成功），刷新页面，继续该教程的剩余部分。

## 例程
以下代码块可以直接运行：

1. 选中下面的代码块
2. 按 Shift + Enter 运行代码块
3. 观看实时视频窗口
4. 按 `STOP` 关闭实时视频，释放摄像头资源

### 如果运行时不能看到摄像头实时画面
- 需要点击上方的 Kernel - Shut down all kernels
- 关闭本章节选项卡，再次打开
- 点击 `STOP` 释放摄像头资源后重新运行代码块
- 重启设备

### 注意事项
如果使用USB摄像头则需要取消注释`frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)`这一句。

### 运行
当代码块运行时，可以通过点击`PHOTO`拍照，通过点击`Video Start`开始录像，通过点击`Video Stop`停止录像。

In [ ]:
import matplotlib.pyplot as plt
import cv2
from picamera2 import Picamera2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading

import os, time

# change the time intervel here (s)
time_intervel = 3 # take a picture every 3 seconds.

# set the dir that pictures and videos save
# you can change this path as you wish
photo_path = '/home/ws/ugv_pt_rpi/static/'
video_path = '/home/ws/ugv_pt_rpi/videos/'

# Stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

# photo take button
# ================
photoButton = widgets.ToggleButton(
    value=False,
    description='Photo',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

# Video Start button
# ================
vstartButton = widgets.ToggleButton(
    value=False,
    description='Video Start',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

# Video Start button
# ================
vstopButton = widgets.ToggleButton(
    value=False,
    description='Video Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)


# Display function
# ================
def view(stop_button, photo_button):
    photo_num_count = 0
    video_num_count = 0
    
    picam2 = Picamera2()
    picam2.configure(picam2.create_video_configuration(main={"format": 'XRGB8888', "size": (640, 480)}))
    picam2.start()
    display_handle=display(None, display_id=True)
    i = 0
    while True:
        frame = picam2.capture_array()
        # frame = cv2.flip(frame, 1) # if your camera reverses your image

        # uncomment this line if you are using USB camera
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # take a picture when photo button pressed.
        if photoButton.value == True:
            photo_num_count = photo_num_count + 1
            photo_filename = f'{photo_path}photo_{num_count}.jpg'
            cv2.imwrite(photo_filename, frame)
            # photoButton.value = False
            print(f'{num_count} photos saved. new photo: {photo_filename}')
            
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            picam2.close()
            display_handle.update(None)

            
# Run
# ================
display(stopButton)
display(photoButton)
thread = threading.Thread(target=view, args=(stopButton, photoButton,))
thread.start()